In [1]:
# Imports
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import undetected_chromedriver as uc

import os
import time
from bs4 import BeautifulSoup
import re
import pandas as pd
import requests

In [2]:
# Configure Chrome Webdriver

chrome_install = ChromeDriverManager().install()

In [ ]:
# Initialize Chrome WebDriver
options = webdriver.ChromeOptions()
#options.add_argument('--headless')
'''
options.add_argument(
    "--user-agent=Mozilla/5.0 (iPhone; CPU iPhone OS 17_0 like Mac OS X) "
    "AppleWebKit/605.1.15 (KHTML, like Gecko) Version/17.0 Mobile/15E148 Safari/604.1"
)
'''
browser = webdriver.Chrome(options=options)

In [4]:
# Setup search parameters
city = "montreal"
city_link = "montréal"
product = "iphone"
min_price = 0
max_price = 5000
days_listed = 60

In [5]:
# Setup base URL
url = f'https://www.facebook.com/marketplace/{city}/search?query={product}&minPrice={min_price}&maxPrice={max_price}&daysSinceListed={days_listed}&exact=false'

# Visit website
browser.get(url)

# Close login pop-up
try:
    close_button = browser.find_element(By.XPATH, "//div[@aria-label='Close']")
    close_button.click()
    print("Login prompt closed successfully...")
except:
    print("Login prompt not closed correctly...")
    pass

Login prompt not closed correctly...


In [6]:
# Scroll down to load all results
last_height = browser.execute_script("return document.body.scrollHeight")

while True:
    # Scroll down to bottom
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(3)
    
    # Calculate new scroll height and compare with last scroll height
    new_height = browser.execute_script("return document.body.scrollHeight")
    if new_height == last_height:
        break
    last_height = new_height
    
    print("Scrolled..")

print("Finished scrolling, all results loaded...")


Scrolled..
Scrolled..
Scrolled..
Scrolled..
Scrolled..
Finished scrolling, all results loaded...


In [7]:
# Retrieve the HTML
html = browser.page_source

soup = BeautifulSoup(html, 'html.parser')

browser.close()

In [8]:
# Find link elements
links = soup.find_all('a')

# Filter on product keyword only
product_links = [
    link for link in links
    if product.lower() in link.text.lower()
    #if city_link.lower() in link.text.lower()
]

product_data = []

for product_link in product_links:
    url = product_link.get('href')
    if not url:
        continue
    text = '\n'.join(product_link.stripped_strings)
    product_data.append({'url': url, 'text': text})

print(len(product_links), "links matched")
print(len(product_data), "items collected")

product_data

162 links matched
162 items collected


[{'url': '/marketplace/item/867133572318716/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$200\niPhone 13 promax\nLongueuil, QC'},
 {'url': '/marketplace/item/882540700769088/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$230\niPhone 11 64 gb\nMont-Tremblant, QC'},
 {'url': '/marketplace/item/1587200549390373/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$380\niPhone 14 simple à vendre\nMontréal, QC'},
 {'url': '/marketplace/item/888959353535453/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$200\niPhone 12\nAlfred and Plantagenet, ON'},
 {'url': '/marketplace/item/1892472571364476/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$550\niPhone 14 Pro\nOttawa, ON'},
 {'url': '/marketplace/item/1761462881237757/?ref=search&referral_code=null&referral_story_type=post&__tn__=!%3AD',
  'text': 'CA$110\niPhone

In [9]:
# Create an empty list to store product data
extracted_data = []

for item in product_data:
    lines = item['text'].split('\n')

    # Regular expression to find numeric values
    numeric_pattern = re.compile('\d[\d,.]*')
    
    
    # Extracting prices
    # Iterate through lines to find the first line with numbers
    for line in lines:
        match = numeric_pattern.search(line)
        if match:    
            # Extract the first numeric value found
            price_str = match.group()
            # Convert price to float (handle commas)
            price = float(price_str.replace(',',''))
            break
            
    if price:
        print(f"Price extracted: {price}")
    else:
        print("price not found")

    # Extract title
    title = lines[-2]

    # Extract location
    location = lines[-1]

    # Add extracted data to a list of dictionaries
    extracted_data.append({
        'title': title,
        'price': price,
        'location': location,
        'url': re.sub(r'\?.*', '', item['url'])
    })

Price extracted: 200.0
Price extracted: 230.0
Price extracted: 380.0
Price extracted: 200.0
Price extracted: 550.0
Price extracted: 110.0
Price extracted: 210.0
Price extracted: 360.0
Price extracted: 1150.0
Price extracted: 685.0
Price extracted: 410.0
Price extracted: 240.0
Price extracted: 250.0
Price extracted: 1500.0
Price extracted: 350.0
Price extracted: 600.0
Price extracted: 650.0
Price extracted: 200.0
Price extracted: 180.0
Price extracted: 300.0
Price extracted: 750.0
Price extracted: 250.0
Price extracted: 380.0
Price extracted: 280.0
Price extracted: 1000.0
Price extracted: 1750.0
Price extracted: 425.0
Price extracted: 280.0
Price extracted: 300.0
Price extracted: 700.0
Price extracted: 800.0
Price extracted: 119.0
Price extracted: 1099.0
Price extracted: 149.0
Price extracted: 239.0
Price extracted: 1350.0
Price extracted: 550.0
Price extracted: 225.0
Price extracted: 600.0
Price extracted: 80.0
Price extracted: 749.0
Price extracted: 299.0
Price extracted: 430.0
Price 

<>:8: SyntaxWarning: invalid escape sequence '\d'
<>:8: SyntaxWarning: invalid escape sequence '\d'
/var/folders/xn/w_xnr1cn1jq5m_6rgnybnsc40000gn/T/ipykernel_3119/1258939502.py:8: SyntaxWarning: invalid escape sequence '\d'
  numeric_pattern = re.compile('\d[\d,.]*')


In [10]:
extracted_data

[{'title': 'iPhone 13 promax',
  'price': 200.0,
  'location': 'Longueuil, QC',
  'url': '/marketplace/item/867133572318716/'},
 {'title': 'iPhone 11 64 gb',
  'price': 230.0,
  'location': 'Mont-Tremblant, QC',
  'url': '/marketplace/item/882540700769088/'},
 {'title': 'iPhone 14 simple à vendre',
  'price': 380.0,
  'location': 'Montréal, QC',
  'url': '/marketplace/item/1587200549390373/'},
 {'title': 'iPhone 12',
  'price': 200.0,
  'location': 'Alfred and Plantagenet, ON',
  'url': '/marketplace/item/888959353535453/'},
 {'title': 'iPhone 14 Pro',
  'price': 550.0,
  'location': 'Ottawa, ON',
  'url': '/marketplace/item/1892472571364476/'},
 {'title': 'iPhone 12 Pro Max',
  'price': 110.0,
  'location': 'Gatineau, QC',
  'url': '/marketplace/item/1761462881237757/'},
 {'title': 'Iphone 11',
  'price': 210.0,
  'location': 'Gatineau, QC',
  'url': '/marketplace/item/798523009849714/'},
 {'title': 'iPhone 12 Pro Max 128GB',
  'price': 360.0,
  'location': 'Brossard, QC',
  'url': '/